In [1]:
# SR as '1' sample
import pandas as pd
SR = {}
# Load spreadsheet
xl = pd.ExcelFile('all_cancer_pair.xlsx')

df = xl.parse(xl.sheet_names[0], skiprows=3)

for index, row in df.iterrows():
# Use the first and second column values as a key, and set the value as 1
    SR[row[0], row[1]] = 1

    
df = xl.parse(xl.sheet_names[2], skiprows=2)

for index, row in df.iterrows():
# Use the first and second column values as a key, and set the value as 1
    SR[row[0], row[1]] = 1

print(len(SR))

/data1/xpgeng/.conda/envs/pytorch/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/data1/xpgeng/.conda/envs/pytorch/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


3000


In [2]:
# Non-SR as '0' sample
from Bio import SeqIO

def extract_gene_symbol(description):
    parts = description.split(' ')
    for part in parts:
        if part.startswith('gene_symbol:'):
            return part.split(':')[1]
    return None

fasta_file = "Homocdsall.fasta"  

cds_dic = {}
for record in SeqIO.parse(fasta_file, "fasta"):
    gene_symbol = extract_gene_symbol(record.description)
    cds_dic[gene_symbol] = str(record.seq)
    
cds_gene_list = []
for k,v in cds_dic.items():       
        cds_gene_list.append(k)
cds_gene_list = list(set(cds_gene_list))
print(len(cds_gene_list))

import random

Non_SR = {}
random.seed(127)
for i in range(30001):
    a = random.randint(0, 19852)
    b = random.randint(0, 19852)
    while a == b:
        b = random.randint(0, 19852)
    if (cds_gene_list[a], cds_gene_list[b]) not in SR:
        
        Non_SR[cds_gene_list[a], cds_gene_list[b]] = 0
    
print(len(Non_SR))

19853
30000


#### DNA 5-mer 

In [3]:
from Bio import SeqIO

def extract_gene_symbol(description):
    parts = description.split(' ')
    for part in parts:
        if part.startswith('gene_symbol:'):
            return part.split(':')[1]
    return None  

fasta_file = "Homocdsall.fasta"  

cds_dic = {}
for record in SeqIO.parse(fasta_file, "fasta"):
    gene_symbol = extract_gene_symbol(record.description)
    cds_dic[gene_symbol] = str(record.seq)
    
# Copy the dictionary to avoid issues with changing the dict size during iteration
cds_dic_copy = cds_dic.copy()

for gene_name, sequence in cds_dic_copy.items():
    for char in sequence:
        if char not in 'ATGC':
            del cds_dic[gene_name]  # Remove this gene from the original dict
            break  # No need to check the rest of the sequence

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

# Function to generate k-mers
def generate_kmers(sequence, k):
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

# Generate 5-mers for each sequence in the dictionary
k = 5
kmers_dic = {gene_name: ' '.join(generate_kmers(seq, k)) for gene_name,
             seq in cds_dic.items()}

# Convert k-mers to feature vectors
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(kmers_dic.values())

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# Function to generate k-mers
def generate_kmers(sequence, k):
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]


# Generate 5-mers for each sequence in the dictionary
k = 5
kmers_dic = {gene_name: ' '.join(generate_kmers(seq, k)) for gene_name, seq in cds_dic.items()}

# Convert k-mers to feature vectors
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(kmers_dic.values())

# Create a dictionary with gene names as keys and feature vectors as values
feature_dic = {gene_name: X[i].toarray()[0] for i, gene_name in enumerate(kmers_dic.keys())}

#### Protein 3-mer PCA 

In [6]:
from Bio import SeqIO

def extract_gene_symbol(description):
    parts = description.split(' ')
    for part in parts:
        if part.startswith('gene_symbol:'):
            return part.split(':')[1]
    return None  

fasta_file = "Homopepall.fasta" 

pep_dic = {}
for record in SeqIO.parse(fasta_file, "fasta"):
    gene_symbol = extract_gene_symbol(record.description)
    pep_dic[gene_symbol] = str(record.seq)

In [7]:
# Copy the dictionary to avoid issues with changing the dict size during iteration
pep_dic_copy = pep_dic.copy()
print(len(pep_dic))
for gene_name, sequence in pep_dic_copy.items():
    for char in sequence:
        if char not in 'ACDEFGHIKLMNPQRSTVWY':
            del pep_dic[gene_name]  # Remove this gene from the original dict
            break  # No need to check the rest of the sequence
            
print(len(pep_dic))

19853
17511


In [8]:
from collections import Counter
from itertools import product
from sklearn.decomposition import PCA
import numpy as np

# Generate all possible 3-mers
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
all_3mers = [''.join(p) for p in product(amino_acids, repeat=3)]

def get_3mer_composition(protein_sequence):
    # Generate 3-mers from the protein sequence
    sequence_3mers = [protein_sequence[i:i+3] for i in range(len(protein_sequence) - 2)]
    
    # Count the frequency of each 3-mer
    counter = Counter(sequence_3mers)
    
    # Create a dictionary with all 3-mers and their frequencies
    composition = {k: counter.get(k, 0) for k in all_3mers}
    
    return composition

# Compute the 3-mer composition for each protein
tri_peptide_composition_dic = {protein_name: get_3mer_composition(seq) for protein_name, seq in pep_dic.items()}

# Create a numpy array of the feature vectors
feature_matrix = np.array([list(dic.values()) for dic in tri_peptide_composition_dic.values()])

# Apply PCA to reduce dimensionality to 1000
pca = PCA(n_components=1000)
reduced_feature_matrix = pca.fit_transform(feature_matrix)

# Map the reduced feature vectors back to the protein names
reduced_feature_dic = {protein_name: reduced_feature_matrix[i] for i, protein_name in enumerate(tri_peptide_composition_dic.keys())}

#### test performance DNA + Protein Sequence

In [9]:
X = []
y = []
from tqdm import tqdm
import numpy as np
for gene_pair, label in tqdm(SR.items()):
    try:       
        a = feature_dic[gene_pair[0]]
        b = feature_dic[gene_pair[1]]
        c = reduced_feature_dic[gene_pair[0]]
        d = reduced_feature_dic[gene_pair[1]]
                
        X.append(np.concatenate([a,b,c,d]))
        y.append(label)        
    except:
        print(gene_pair)
        
for gene_pair, label in tqdm(Non_SR.items()):
    try:       
        a = feature_dic[gene_pair[0]]
        b = feature_dic[gene_pair[1]]
        c = reduced_feature_dic[gene_pair[0]]
        d = reduced_feature_dic[gene_pair[1]]
                
        X.append(np.concatenate([a,b,c,d]))
        y.append(label)        
    except:
        print(gene_pair)

100%|██████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 41683.08it/s]


('ABL1', 'MAST1')
('ACCS', 'XRCC5')
('ADCK5', 'RNMT')
('ADPRHL2', 'BICC1')
('ADPRHL2', 'FUT11')
('ADPRHL2', 'IPMK')
('AHR', 'RET')
('AKIRIN1', 'MAST1')
('AKT1', 'EGFR')
('ANKRD9', 'MAST1')
('ANLN', 'CGNL1')
('ANTXR2', 'NT5M')
('ARHGAP29', 'MRP63')
('ARHGEF1', 'KIAA2018')
('ATIC', 'MAPK1')
('ATIC', 'AURKC')
('ATM', 'DNMT1')
('AURKA', 'MAST1')
('B3GNT4', 'KIAA1704')
('B4GALT2', 'KIAA1704')
('BCAR1', 'PDPK1')
('BCL11A', 'AMFR')
('BCL2L2', 'DNMT1')
('BGN', 'ADAMTS18')
('BIRC5', 'BRAF')
('BIRC5', 'CLMN')
('BIRC5', 'CYP4X1')
('BIRC5', 'NFE2L2')
('BIRC5', 'ZNF415')
('BMP8A', 'IGF1R')
('BMP8B', 'ADAMTS17')
('BRD4', 'DDR1')
('BRE', 'AMFR')
('BUB1', 'EPHX2')
('BUB1', 'FAM149A')
('BUB1', 'OBFC1')
('BUB1B', 'SEPP1')
('BUB1B', 'ATXN7L1')
('C12orf52', 'AKAP11')
('C14orf179', 'MAST1')
('C17orf53', 'CDKN2AIP')
('C17orf53', 'MAST4')
('C4orf32', 'LSR')
('C7', 'EME1')
('CALD1', 'HMGA1')
('CALD1', 'RASSF7')
('CARD6', 'EME1')
('CCDC59', 'ZFYVE20')
('CCM2', 'MED13L')
('CCNE1', 'MPP5')
('CCNF', 'CD302')
('CC

  8%|████▏                                                | 2344/30000 [00:00<00:01, 23434.16it/s]

('PLEKHG7', 'RASAL3')
('OR6C74', 'DNAJC16')
('TMEM163', 'ARMC6')
('TNFSF12-TNFSF13', 'HR')
('SPDYE4', 'CIDEC')
('TRAJ53', 'RAB17')
('MAGEA1', 'RYBP')
('KRTAP9-4', 'SLC26A11')
('DVL1', 'METTL25B')
('RETREG2', 'ACVR2B')
('METTL25B', 'PRDX2')
('ANP32E', 'SELENOO')
('MCM4', 'NBEAL1')
('IVD', 'PATZ1')
('TRABD2B', 'MARK3')
('HMGXB3', 'FAM114A1')
('MSRA', 'PTDSS2')
('PGC', 'DECR1')
('RNASEK-C17orf49', 'DDX1')
('ZIC1', 'HLA-DPB1')
('MT1G', 'SNX33')
('BBIP1', 'MPDZ')
('VSTM5', 'RUVBL1')
('SURF4', 'PLB1')
('FOXI2', 'INO80E')
('S100A11', 'SLC44A2')
('UNC5A', 'CNNM1')
('GPR15', 'BUB1B')
('DPY19L4', 'PHLDA1')
('PLXNA3', 'MYL4')
('ZIC2', 'FCGR2A')
('GJB5', 'SPPL2B')
('RYBP', 'JSRP1')
('RHEX', 'LRRK2')
('KNL1', 'REL')
('KRTAP10-11', 'POTEA')
('GPR157', 'TRIT1')
('IL7', 'CRTC3')
('FKBP4', 'KIF20A')
('CLRN2', 'WDR25')
('GARIN2', 'ANKRD18A')
('IGFBP1', 'FDFT1')
('CD200', 'SREK1')
('FBXL17', 'RNF34')
('HSPE1', 'PCIF1')
('ZNF678', 'NUTM2D')
('GPR32', 'DNHD1')
('LMBRD1', 'RAB36')
('EXOG', 'GPR33')
('NLGN3'

 16%|████████▍                                            | 4774/30000 [00:00<00:01, 23934.44it/s]

...

('SYNRG', 'FDFT1')
('RPL7L1', 'COA1')
('PRDM15', 'RAD54L')
('IZUMO2', 'MRPL34')
('GOLGA6L4', 'TTLL4')
('RBM17', 'RAB20')
('TRBJ1-1', 'BAG5')
('LARP6', 'RCC2')
('ANKRD24', 'NSUN6')
('SH3TC1', 'GPR171')
('NCOA5', 'IGKJ2')
('IGHV3-66', 'DDX27')
('TRIM32', 'NEB')
('WDR18', 'RNASE2')
('TRAJ56', 'LMO2')
('TRPM4', 'PSTK')
('GGT6', 'BTBD1')
('AGFG2', 'MYH6')
('ZC3H13', 'HRC')
('SELENOH', 'TADA2A')
('AGBL4', 'MAIP1')
('RBM26', 'VEGFB')
('CEP164', 'PCSK9')
('SPRY1', 'TMEM191C')
('APP', 'ZSCAN5C')
('FAM246C', 'SELENOF')
('C2orf49', 'LAMC3')
('NTNG2', 'TM6SF1')
('ANKRD18B', 'IL36A')
('SLIRP', 'CD37')
('FAM83H', 'FAM221B')
('COX5A', 'PTBP1')
('ANHX', 'GRAMD4')
('NET1', 'GALNT12')
('ZNF383', 'OPN1MW')
('PPIL1', 'SPDYE4')
('PPIH', 'ADAMTS18')
('IRS4', 'SCNN1B')
('MARCHF2', 'CSNK1G2')
('ANTKMT', 'CPSF4')
('FAM149A', 'CBY2')
('TRAF2', 'NYAP2')
('MTFP1', 'PDHX')
('AP1AR', 'SS18L1')
('ZC3H8', 'SLC12A5')
('PTPN18', 'VIT')
('RBBP6', 'PTGDR2')
('RGS14', 'H3C11')
('CFAP97D1', 'RTCA')
('RBCK1', 'CEP290')
('SE

 87%|█████████████████████████████████████████████▎      | 26166/30000 [00:01<00:00, 23793.74it/s]

('MAP4', 'MOG')
('GIT1', 'SLC23A2')
('CLIP2', 'UBE2T')
('POFUT2', 'PSMC4')
('FLT3LG', 'LILRB3')
('DIO3', 'SGK1')
('NNT', 'RO60')
('VGLL2', 'GNAL')
('TNFRSF17', 'TMEM9')
('EMSY', 'MYOG')
('NCAPG2', 'MYL4')
('PABPC1L', 'CYTL1')
('L1TD1', 'NELFA')
('ATP11B', 'MED17')
('SCFD1', 'TREX2')
('PACS1', 'GIPR')
('PTGDS', 'OXER1')
('CEP43', 'TCEANC')
('ANXA8', 'LARS2')
('PHKB', 'ZNF334')
('SLC35E2B', 'QSER1')
('RAB35', 'TRBJ2-2')
('PEPD', 'POU5F1')
('TTC36', 'NLRC5')
('IL12RB2', 'NPFFR1')
('MARS1', 'TFAP2A')
('MDM4', 'KRT72')
('CEACAM21', 'UBAP1L')
('GPRIN3', 'TIMM21')
('TRBC2', 'ST6GALNAC1')
('SOAT2', 'R3HDM1')
('FOXK1', 'NOX3')
('C1QL2', 'PIERCE1')
('CDCA5', 'FAM25C')
('B3GALNT2', 'CPLANE2')
('MGA', 'MYCBP2')
('RNASE2', 'CHURC1')
('SMLR1', 'NOP9')
('CPNE7', 'DLX6')
('NPNT', 'CLHC1')
('SGF29', 'TEP1')
('PCNX4', 'ZNF268')
('MT-CO1', 'TBL1Y')
('TAS1R1', 'GPT')
('MTHFD1', 'HELB')
('AVEN', 'NOTCH1')
('CLN5', 'IPPK')
('NPAS4', 'SLC25A10')
('NRXN1', 'ARPC5')
('IDE', 'STPG1')
('SELENOV', 'VWA5B2')
('ANT

100%|████████████████████████████████████████████████████| 30000/30000 [00:01<00:00, 22876.96it/s]

('CCNG1', 'BTF3')
('QPCTL', 'SHOX2')
('UMODL1', 'SYNRG')
('TP53RK', 'TBC1D24')
('WFDC3', 'MUC16')
('FGF21', 'SOCS7')
('HHLA1', 'MLEC')
('TMEM255B', 'KRBA1')
('PCDHB16', 'ANTKMT')
('TRAJ49', 'TNNC1')
('C1orf167', 'GFPT2')
('CHD5', 'EIF2AK3')
('CDC25B', 'TAF8')
('SFXN1', 'OR2J1')
('QKI', 'EME1')
('KANSL1', 'SERINC4')
('FOXR2', 'FTO')
('ZC3H7B', 'COL13A1')
('FKBP10', 'SERPINA9')
('RGSL1', 'CCNI2')
('GAB1', 'TRIM50')
('PUM1', 'BCAM')
('DDX46', 'LIPC')
('DENND4C', 'BABAM2')
('SPNS1', 'DYNLT2B')
('CACNA1E', 'TSR3')
('ZFAND2B', 'DYNC1H1')
('TEPSIN', 'PDGFA')
('POLR1C', 'COX4I1')
('NRBP2', 'LGALS14')
('CS', 'TMEM219')
('POLR1C', 'IRF1')
('TNFSF10', 'CELSR3')
('PPIAL4G', 'ST8SIA3')
('PLCH1', 'REEP2')
('ERC2', 'GLMN')
('RIPK2', 'VAV3')
('APLP1', 'DENND3')
('NUBP1', 'CFHR5')
('TSN', 'FAM246C')
('KRTAP23-1', 'TRAJ13')
('DIRAS3', 'ARHGEF33')
('PLSCR5', 'C8G')
('NR1H4', 'VSIR')
('RNF167', 'LCN8')
('PRMT2', 'TRAJ13')
('AP2A1', 'SLC1A5')
('ZBTB7A', 'TNS2')
('OR4K3', 'VDR')
('CD276', 'MUSK')
('GLT1D1',

Gaussian Naive Bayes

In [12]:
import random
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

# 合并X和y
data = list(zip(X, y))

# 打乱数据
random.shuffle(data)

# 将数据分成五部分
n = len(data) // 5
data_splits = [data[i * n:(i + 1) * n] for i in range(5)]

# 保存所有的AUC
auc_scores = []

# 初始化高斯朴素贝叶斯模型
gnb = GaussianNB()

# 对每个分割，使用它作为测试集，其余的作为训练集
for i in range(5):
    print(i)
    test_data = data_splits[i]
    train_data = [item for sublist in data_splits[:i] + data_splits[i + 1:] for item in sublist]

    # 分离X和y
    X_test, y_test = zip(*test_data)
    X_train, y_train = zip(*train_data)

    # 训练模型
    gnb.fit(X_train, y_train)

    # 在测试集上预测
    y_pred = gnb.predict(X_test)

    # 计算并保存AUC
    auc = roc_auc_score(y_test, y_pred)
    auc_scores.append(auc)

# 计算平均AUC
average_auc = sum(auc_scores) / len(auc_scores)
print("Average AUC:", average_auc)

0
1
2
3
4
Average AUC: 0.5790101965251047


Gradient Boosting

In [13]:
import random
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

# 合并X和y
data = list(zip(X, y))

# 打乱数据
random.shuffle(data)

# 将数据分成五部分
n = len(data) // 5
data_splits = [data[i * n:(i + 1) * n] for i in range(5)]

# 保存所有的AUC
auc_scores = []

# 初始化梯度提升模型
gb = GradientBoostingClassifier()

# 对每个分割，使用它作为测试集，其余的作为训练集
for i in range(5):
    print(i)
    test_data = data_splits[i]
    train_data = [item for sublist in data_splits[:i] + data_splits[i + 1:] for item in sublist]

    # 分离X和y
    X_test, y_test = zip(*test_data)
    X_train, y_train = zip(*train_data)

    # 训练模型
    gb.fit(X_train, y_train)

    # 在测试集上预测
    y_pred = gb.predict(X_test)

    # 计算并保存AUC
    auc = roc_auc_score(y_test, y_pred)
    auc_scores.append(auc)

# 计算平均AUC
average_auc = sum(auc_scores) / len(auc_scores)
print("Average AUC:", average_auc)

0
1
2
3
4
Average AUC: 0.6527832685639046


Random Forest

In [14]:
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# 合并X和y
data = list(zip(X, y))

# 打乱数据
random.shuffle(data)

# 将数据分成五部分
n = len(data) // 5
data_splits = [data[i * n:(i + 1) * n] for i in range(5)]

# 保存所有的AUC
auc_scores = []

# 初始化随机森林模型
rf = RandomForestClassifier()

# 对每个分割，使用它作为测试集，其余的作为训练集
for i in range(5):
    test_data = data_splits[i]
    train_data = [item for sublist in data_splits[:i] + data_splits[i + 1:] for item in sublist]

    # 分离X和y
    X_test, y_test = zip(*test_data)
    X_train, y_train = zip(*train_data)

    # 训练模型
    rf.fit(X_train, y_train)

    # 在测试集上预测
    y_pred = rf.predict(X_test)

    # 计算并保存AUC
    auc = roc_auc_score(y_test, y_pred)
    auc_scores.append(auc)

# 计算平均AUC
average_auc = sum(auc_scores) / len(auc_scores)
print("Average AUC:", average_auc)

Average AUC: 0.7178550946621349


SVM

In [15]:
import random
from sklearn import svm
from sklearn.metrics import roc_auc_score

# 合并X和y
data = list(zip(X, y))

# 打乱数据
random.shuffle(data)

# 将数据分成五部分
n = len(data) // 5
data_splits = [data[i * n:(i + 1) * n] for i in range(5)]

# 保存所有的AUC
auc_scores = []

# 初始化SVM模型
svc = svm.SVC(probability=True)

# 对每个分割，使用它作为测试集，其余的作为训练集
for i in range(5):
    test_data = data_splits[i]
    train_data = [item for sublist in data_splits[:i] + data_splits[i + 1:] for item in sublist]

    # 分离X和y
    X_test, y_test = zip(*test_data)
    X_train, y_train = zip(*train_data)

    # 训练模型
    svc.fit(X_train, y_train)

    # 在测试集上预测
    y_pred = svc.predict_proba(X_test)[:, 1]

    # 计算并保存AUC
    auc = roc_auc_score(y_test, y_pred)
    auc_scores.append(auc)

# 计算平均AUC
average_auc = sum(auc_scores) / len(auc_scores)
print("Average AUC:", average_auc)

Average AUC: 0.8521987037166555
